<a href="https://colab.research.google.com/github/milkywayisouruniverse/MNIST/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn plotly



In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten

# Load data
(x_train, y_train), (_, _) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)

# Build classifier with feature layer
inputs = Input(shape=(28,28,1))
x = Flatten()(inputs)
x = Dense(128, activation='relu')(x)
features = Dense(32, activation='relu')(x)
outputs = Dense(10, activation='softmax')(features)

model = Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=256)


Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7544 - loss: 0.8567
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9436 - loss: 0.1988
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9622 - loss: 0.1343
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9685 - loss: 0.1098
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9749 - loss: 0.0834


In [4]:
# Create feature extractor model
feature_model = Model(inputs=model.input,
                      outputs=model.layers[-2].output)

feat = feature_model.predict(x_train)

print(feat.shape)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
(60000, 32)


In [5]:
!pip install umap-learn


In [6]:
import umap

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1)
proj = reducer.fit_transform(feat)

print(proj.shape)


(60000, 2)


In [7]:
!pip install plotly


In [8]:
import plotly.express as px

fig = px.scatter(
    x=proj[:,0],
    y=proj[:,1],
    color=y_train.astype(str),
    title="MNIST Neural Feature Space",
    labels={'color': 'Digit'}
)

fig.update_traces(marker=dict(size=3))
fig.show()


In [10]:
import plotly.express as px

fig = px.scatter(
    x=proj[:,0],
    y=proj[:,1],
    color=y_train.astype(str),
    title="UMAP Projection"
)

fig.update_traces(marker=dict(size=3))
fig.show()

In [12]:
from sklearn.manifold import TSNE

# For t-SNE, it's often better to use a subset of data due to computational cost
# We'll use the first 10,000 samples, matching the y_train slicing in the plot
tsne_reducer = TSNE(n_components=2, random_state=0)
tsne_proj = tsne_reducer.fit_transform(feat[:10000])

fig = px.scatter(
    x=tsne_proj[:,0],
    y=tsne_proj[:,1],
    color=y_train[:10000].astype(str),
    title="t-SNE Projection"
)

fig.update_traces(marker=dict(size=3))
fig.show()

In [14]:
fig = px.scatter(
    x=proj[:,0],
    y=proj[:,1],
    color=y_train.astype(str),
    title="UMAP Projection"
)

fig.update_traces(marker=dict(size=3))
fig.show()